In [7]:
import os
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow.keras import models
import tensorflow_addons as tfa

C:\Program Files\Python39\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [51]:
class_index = {0: 'angry', 1: 'fear', 2: 'happy',3: 'neutral', 4: 'sad'}
filenames_with_path = tf.io.gfile.glob('data/test/*')
filenames = os.listdir('data/test/')
sample_rate = 16000


@tf.function(jit_compile=False)
def preprocess_val(filepath):
      wav = tf.io.read_file(filepath)
      wav, sr = tf.audio.decode_wav(wav, desired_channels=1)
      wav = tf.squeeze(wav, axis=-1)
      if no_norm: wav *= 32768.0
      sr = tf.cast(sr, dtype=tf.int64)
      wav = tfio.audio.resample(wav, rate_in=sr, rate_out=sample_rate)
      spectrogram = tfio.audio.spectrogram(wav, nfft=512, window=512, stride=256)
      mel_spectrogram = tfio.audio.melscale(spectrogram, rate=sample_rate, mels=128, fmin=80, fmax=7600)

      # Augmentations
      # mel_spectrogram = tfio.audio.freq_mask(mel_spectrogram, param=10)
      # mel_spectrogram = tfio.audio.time_mask(mel_spectrogram, param=10)

      # Resizing
      mel_spectrogram = tf.expand_dims(mel_spectrogram, axis=-1)
      mel_spectrogram = tf.image.resize_with_pad(mel_spectrogram, target_height=224, target_width=224, method=tf.image.ResizeMethod.BILINEAR)  # any sharper resizing method produce black gaps
      mel_spectrogram = tf.squeeze(tf.stack([mel_spectrogram, mel_spectrogram, mel_spectrogram], axis=-1))
      mel_spectrogram.set_shape([224, 224, 3])
      return mel_spectrogram


no_norm = True
test_ds = tf.data.Dataset.from_tensor_slices(filenames_with_path).map(preprocess_val, num_parallel_calls=-1)
test_ds = test_ds.batch(32).cache().prefetch(-1)

In [8]:
opt = tfa.optimizers.AdamW(learning_rate=1e-3, weight_decay=5e-5)
opt = tfa.optimizers.Lookahead(opt)

model1 = models.load_model('models/regnetx016_19epoch_7813_ranger/', compile=False, custom_objects={'Addons>Lookahead': opt})
model2 = models.load_model('models/regnetx016_22epoch_7813/', compile=False, custom_objects={'Addons>Lookahead': opt})  # this one seems worse

In [52]:
preds1 = model1.predict(test_ds)
preds1 = tf.nn.softmax(preds1, axis=-1)
preds2 = model2.predict(test_ds)
preds2 = tf.nn.softmax(preds2, axis=-1)

63/63 [==============================] - 2s 28ms/step


In [71]:
pred = 0.5 * preds1 + 0.5 * preds2
pred_classes = tf.argmax(pred, axis=-1).numpy()
pred_classes = [class_index[i] for i in pred_classes]
pred_df = pd.DataFrame({'filename': filenames, 'pred': pred_classes})
pred_df.to_csv('submission.csv', index=False)

In [72]:
print(f"Accuracy: {(1 - (len(pred_df[pred_df['pred'] != 'angry']) / len(pred_df))) * 100}%")

Accuracy: 97.2554890219561%
